# **Setup**

In [ ]:
import wandb
wandb.login()

In [1]:
import torch
from torch.utils.data import DataLoader
#from torchsummary import summary
import gc
import h5py
from UNet import UNet, Encoder, ContrastiveEncoder, ResNetBlock
from LITSDataset import LITSBinaryDataset, LITSContDataset
import LossFunctions
import TrainingEval
from tqdm import tqdm

In [2]:
#Hyperparameters and training modifications
device = torch.device("cuda") if torch.cuda.is_available() else torch.device("cpu")
#device = torch.device("cpu")

modelName = "ResUNetTest"
modelFile = "UsedModels/" + modelName
configFile = "testConfig.txt"

startEpoch = 0
useWandB = 0
batchSize = 0
learnRate = 0
epochs = 0
startDim = 0
epochsToDouble = 0
progressive = 0
epochsToSave = 0
cosineAnnealing = 0
cosineRestartEpochs = 0

varDict = {
    "startEpoch":startEpoch,
    "useWandB":useWandB,
    "batchSize":batchSize,
    "learnRate":learnRate,
    "epochs":epochs,
    "startDim":startDim,
    "epochsToDouble":epochsToDouble,
    "progressive":progressive,
    "epochsToSave":epochsToSave,
    "cosineAnnealing":cosineAnnealing,
    "cosineRestartEpochs":cosineRestartEpochs,
}

TrainingEval.ParseConfig(configFile, varDict)

for key in varDict:
    if varDict[key].is_integer():
        locals()[key] = int(varDict[key])
    else:
        locals()[key] = varDict[key]

In [3]:
#Load Datasets
#trainDataset = LITSBinaryDataset("Datasets/StandardDatasets/FullTrainDataset.hdf5")
trainDataset = LITSBinaryDataset("Datasets/StandardDatasets/ReducedTrainingDataset.hdf5")
validationDataset = LITSBinaryDataset("Datasets/StandardDatasets/ValidationDataset.hdf5")
testDataset = LITSBinaryDataset("Datasets/StandardDatasets/TestDataset.hdf5")

trainIter = DataLoader(trainDataset, batch_size=batchSize, shuffle=True)
validationIter = DataLoader(validationDataset, batch_size=batchSize)
testIter = DataLoader(testDataset, batch_size=batchSize)

print("Datasets loaded")

Datasets loaded


# **Standard Training**

In [ ]:
#Load model
initModel = ""

lossFuncs = [[LossFunctions.dice_loss, LossFunctions.dice_score], []]
weights = [[1, 0], []]

segmenter = UNet(1, block=ResNetBlock)
#print(summary(net, (1, 256, 256)))

#Loads model from file if using a pretrained version
if initModel != "":
    segmenter.load_state_dict(torch.load(initModel))

segmenter = segmenter.to(device)

print("Intialized standard UNet model")

In [ ]:
#Train Model

if useWandB:
    wandb.init(project="LiverSegmentation",
            name=modelName,
            config={
                "BatchSize":batchSize,
                "LearnRate":learnRate,
                "Epochs":epochs,
                "StartDimension":startDim,
                "EpochsToDouble":epochsToDouble
            })

TrainingEval.train(segmenter, lossFuncs, weights, trainIter, validationIter, epochs, startEpoch, learnRate, device, startDim, epochsToDouble, modelFile, epochsToSave, useWandB=useWandB, 
      cosineAnnealing=cosineAnnealing, restartEpochs=cosineRestartEpochs, progressive=progressive)

if useWandB:
    wandb.finish()

# **Pre-Training**

In [ ]:
focal = LossFunctions.FocalLoss(weight0=0.1, weight1=0.9, gamma=2)

lossFuncs = [[], [focal, LossFunctions.accuracy, LossFunctions.f1]]
weights = [[], [1, 0, 0]]

initEncoder = "Run 2/Progressive Encoders/ProgEncoder10"
encoderFile = "UsedModels/Encoder1"

encoder = Encoder(1)

if initEncoder != "":
    encoder.load_state_dict(torch.load(initEncoder))

encoder = encoder.to(device)

In [ ]:
#Train Model
gc.collect()

if useWandB:
    wandb.init(project="PreTrainedEncoder",
            name="UNetEncoder",
            config={
                "BatchSize":batchSize,
                "LearnRate":learnRate,
                "Epochs":epochs,
                "StartDimension":startDim,
                "EpochsToDouble":epochsToDouble,
            })

print(TrainingEval.train(encoder, lossFuncs, weights, trainIter, validationIter, epochs, startEpoch, learnRate, device, startDim, epochsToDouble, encoderFile, epochsToSave, useWandB=useWandB, 
      cosineAnnealing=cosineAnnealing, restartEpochs=cosineRestartEpochs, progressive=progressive, encoder=True))

if useWandB:
    wandb.finish()

In [ ]:
segmenter = UNet(encoder=encoder)

lossFuncs = [[LossFunctions.dice_loss, LossFunctions.dice_score], []]
weights = [[1, 0], []]

In [ ]:
#Train Model
gc.collect()

if useWandB:
    wandb.init(project="LiverSegmentationPreTraining",
            name="NoWeights",
            config={
                "BatchSize":batchSize,
                "LearnRate":learnRate,
                "Epochs":epochs,
                "StartDimension":startDim,
                "EpochsToDouble":epochsToDouble
            })

TrainingEval.train(segmenter, lossFuncs, weights, trainIter, validationIter, epochs, startEpoch, learnRate, device, startDim, epochsToDouble, modelFile, epochsToSave, useWandB=useWandB, 
      cosineAnnealing=cosineAnnealing, restartEpochs=cosineRestartEpochs, progressive=0)

if useWandB:
    wandb.finish()

# **Joint Training**

In [ ]:
for i in tqdm(range(10)):
    modelFile = "ContJoint" + str(i)
    initEncoder = "ContrastiveModels/Encoders/ContrastiveEncoder" + str(i)

    encoder = Encoder()
    encoder.load_state_dict(torch.load(initEncoder), strict=False)

    segmenter = UNet(device, encoder=encoder)

    if useWandB:
        wandb.init(project="LiverSegmentationJointTraining",
                name="Weights:",
                config={
                    "BatchSize":batchSize,
                    "LearnRate":learnRate,
                    "Epochs":epochs,
                    "StartDimension":startDim,
                    "EpochsToDouble":epochsToDouble
                })

    classLossFunc = LossFunctions.FocalLoss(weight0=0.2, weight1=0.8, gamma=2)

    lossFuncs = [[LossFunctions.dice_score, LossFunctions.dice_loss], [LossFunctions.accuracy, classLossFunc]]
    weights = [[0, 0.6], [0, 0.4]]

    TrainingEval.train(segmenter, lossFuncs, weights, trainIter, validationIter, epochs, startEpoch, learnRate, device, startDim, epochsToDouble, modelFile, epochsToSave, useWandB=useWandB, 
        cosineAnnealing=cosineAnnealing, restartEpochs=cosineRestartEpochs, progressive=progressive)

    if useWandB:
        wandb.finish()

# **Contrastive Pre-Training**

In [ ]:
contTrainDataset = LITSContDataset("Datasets/ContrastiveDatasets/ScanBased/RandContrastiveTrainDataset.hdf5")
contValDataset = LITSContDataset("Datasets/ContrastiveDatasets/ScanBased/RandContrastiveValidationDataset.hdf5")
contTestDataset = LITSContDataset("Datasets/ContrastiveDatasets/ScanBased/RandContrastiveTestDataset.hdf5")

contTrainIter = DataLoader(contTrainDataset, batch_size=batchSize, shuffle=True)
contValidationIter = DataLoader(contValDataset, batch_size=batchSize)
contTestIter = DataLoader(contTestDataset, batch_size=batchSize)

In [ ]:
for i in tqdm(range(10)):
    modelName = "RandContrastiveEncoder" + str(i)
    modelFile = "UsedModels/" + modelName
    encoder = ContrastiveEncoder()

    if useWandB:
        wandb.init(project="LITSEncoderContrastive",
                name="Weights:",
                config={
                    "BatchSize":batchSize,
                    "LearnRate":learnRate,
                    "Epochs":epochs,
                    "StartDimension":startDim,
                    "EpochsToDouble":epochsToDouble
                })
        
    #lossFunc = LossFunctions.ContrastiveLossEuclidean
    lossFunc = LossFunctions.ContrastiveLossCosine(temp=(1 / batchSize))

    TrainingEval.contrastiveTrain(encoder, lossFunc, contTrainIter, contValidationIter, epochs, startEpoch, learnRate, device, modelFile, epochsToSave, useWandB=useWandB, 
        cosineAnnealing=cosineAnnealing, restartEpochs=cosineRestartEpochs, isDist=False)

    if useWandB:
        wandb.finish()

    print("Model: " + str(i) + " finished")

In [ ]:
initEncoder = "ContrastiveTest/ContrastiveTest1Encoder"

encoder = ContrastiveEncoder()
encoder.load_state_dict(torch.load(initEncoder), strict=False)

segmenter = UNet(device, encoder=encoder)
#segmenter.freezeEncoder()

lossFuncs = [[LossFunctions.dice_loss, LossFunctions.dice_score], []]
weights = [[1, 0], []]

In [4]:
for i in tqdm(range(10)):
    initEncoder = "ContrastiveModels/ScanBased/Encoders/RandContrastiveEncoder" + str(i)
    modelFile = "UsedModels/RandContReducedFineTune" + str(i)

    print(modelFile)

    encoder = ContrastiveEncoder()
    encoder.load_state_dict(torch.load(initEncoder), strict=False)

    segmenter = UNet(device, encoder=encoder)
    #segmenter.freezeEncoder()

    lossFuncs = [[LossFunctions.dice_loss, LossFunctions.dice_score], []]
    weights = [[1, 0], []]

    #Train Model
    gc.collect()

    if useWandB:
        wandb.init(project="LiverSegmentationPreTraining",
                name="NoWeights",
                config={
                    "BatchSize":batchSize,
                    "LearnRate":learnRate,
                    "Epochs":epochs,
                    "StartDimension":startDim,
                    "EpochsToDouble":epochsToDouble
                })

    TrainingEval.train(segmenter, lossFuncs, weights, trainIter, validationIter, epochs, startEpoch, learnRate, device, startDim, epochsToDouble, modelFile, epochsToSave, useWandB=useWandB, 
        cosineAnnealing=cosineAnnealing, restartEpochs=cosineRestartEpochs, progressive=0)

    if useWandB:
        wandb.finish()

  0%|          | 0/10 [00:00<?, ?it/s]

UsedModels/RandContReducedFineTune0
Training on cuda
Epoch 0:
Train Loss: 0.9257338643074036 Validation Loss: 0.9830759167671204 dice_loss: tensor(0.9831, device='cuda:0') dice_score: tensor(0.0496, device='cuda:0') 
Epoch 1:
Train Loss: 0.9114002585411072 Validation Loss: 0.980453610420227 dice_loss: tensor(0.9805, device='cuda:0') dice_score: tensor(0.0693, device='cuda:0') 
Epoch 2:
Train Loss: 0.9027789235115051 Validation Loss: 0.9774364829063416 dice_loss: tensor(0.9774, device='cuda:0') dice_score: tensor(0.0746, device='cuda:0') 
Epoch 3:
Train Loss: 0.8915013670921326 Validation Loss: 0.9740831255912781 dice_loss: tensor(0.9741, device='cuda:0') dice_score: tensor(0.1058, device='cuda:0') 
Epoch 4:
Train Loss: 0.8858792185783386 Validation Loss: 0.9715023040771484 dice_loss: tensor(0.9715, device='cuda:0') dice_score: tensor(0.1016, device='cuda:0') 
Epoch 5:
Train Loss: 0.8786389231681824 Validation Loss: 0.9696374535560608 dice_loss: tensor(0.9696, device='cuda:0') dice_scor

 10%|█         | 1/10 [21:48<3:16:15, 1308.37s/it]

Epoch 99:
Train Loss: 0.6896054148674011 Validation Loss: 0.7847359776496887 dice_loss: tensor(0.7847, device='cuda:0') dice_score: tensor(0.2180, device='cuda:0') 
UsedModels/RandContReducedFineTune1
Training on cuda
Epoch 0:
Train Loss: 0.9272376894950867 Validation Loss: 0.9833638668060303 dice_loss: tensor(0.9834, device='cuda:0') dice_score: tensor(0.0463, device='cuda:0') 
Epoch 1:
Train Loss: 0.914934515953064 Validation Loss: 0.9809331297874451 dice_loss: tensor(0.9809, device='cuda:0') dice_score: tensor(0.0431, device='cuda:0') 
Epoch 2:
Train Loss: 0.9008979797363281 Validation Loss: 0.9771259427070618 dice_loss: tensor(0.9771, device='cuda:0') dice_score: tensor(0.0617, device='cuda:0') 
Epoch 3:
Train Loss: 0.8894011378288269 Validation Loss: 0.9734086990356445 dice_loss: tensor(0.9734, device='cuda:0') dice_score: tensor(0.0734, device='cuda:0') 
Epoch 4:
Train Loss: 0.8807116746902466 Validation Loss: 0.9699051976203918 dice_loss: tensor(0.9699, device='cuda:0') dice_sco

 20%|██        | 2/10 [43:27<2:53:41, 1302.75s/it]

...

Epoch 99:
Train Loss: 0.6901904344558716 Validation Loss: 0.785243034362793 dice_loss: tensor(0.7852, device='cuda:0') dice_score: tensor(0.2167, device='cuda:0') 
UsedModels/RandContReducedFineTune8
Training on cuda
Epoch 0:
Train Loss: 0.9297113418579102 Validation Loss: 0.983731746673584 dice_loss: tensor(0.9837, device='cuda:0') dice_score: tensor(0.0188, device='cuda:0') 
Epoch 1:
Train Loss: 0.9137283563613892 Validation Loss: 0.9814773201942444 dice_loss: tensor(0.9815, device='cuda:0') dice_score: tensor(0.0296, device='cuda:0') 
Epoch 2:
Train Loss: 0.9028337597846985 Validation Loss: 0.9774923920631409 dice_loss: tensor(0.9775, device='cuda:0') dice_score: tensor(0.0462, device='cuda:0') 
Epoch 3:
Train Loss: 0.8922795653343201 Validation Loss: 0.9745252132415771 dice_loss: tensor(0.9745, device='cuda:0') dice_score: tensor(0.0490, device='cuda:0') 
Epoch 4:
Train Loss: 0.8831285834312439 Validation Loss: 0.9719060659408569 dice_loss: tensor(0.9719, device='cuda:0') dice_scor

 90%|█████████ | 9/10 [3:15:08<21:41, 1301.13s/it]

Epoch 99:
Train Loss: 0.6704109311103821 Validation Loss: 0.6314865350723267 dice_loss: tensor(0.6315, device='cuda:0') dice_score: tensor(0.6074, device='cuda:0') 
UsedModels/RandContReducedFineTune9
Training on cuda
Epoch 0:
Train Loss: 0.9242058396339417 Validation Loss: 0.9838382005691528 dice_loss: tensor(0.9838, device='cuda:0') dice_score: tensor(0.0268, device='cuda:0') 
Epoch 1:
Train Loss: 0.9104936122894287 Validation Loss: 0.9810023307800293 dice_loss: tensor(0.9810, device='cuda:0') dice_score: tensor(0.0348, device='cuda:0') 
Epoch 2:
Train Loss: 0.9011568427085876 Validation Loss: 0.9760518074035645 dice_loss: tensor(0.9761, device='cuda:0') dice_score: tensor(0.0640, device='cuda:0') 
Epoch 3:
Train Loss: 0.8869754076004028 Validation Loss: 0.9710426926612854 dice_loss: tensor(0.9710, device='cuda:0') dice_score: tensor(0.0913, device='cuda:0') 
Epoch 4:
Train Loss: 0.8777115941047668 Validation Loss: 0.9666044116020203 dice_loss: tensor(0.9666, device='cuda:0') dice_sc

100%|██████████| 10/10 [3:36:47<00:00, 1300.71s/it]

Epoch 99:
Train Loss: 0.6879538297653198 Validation Loss: 0.80277419090271 dice_loss: tensor(0.8028, device='cuda:0') dice_score: tensor(0.1988, device='cuda:0') 


# **Evaluation/Ending**

In [ ]:
for i in range(10):
    modelName = "ContJoint" + str(i)
    classification = False
    modelFile = "UsedModels/RandContReducedFineTune" + str(i) + "BestLoss"
    classLossFunc = LossFunctions.FocalLoss(weight0=0.2, weight1=0.8, gamma=2)

    lossFuncs = [[LossFunctions.dice_score, LossFunctions.hausdorff], []]
    #lossFuncs = [[LossFunctions.dice_score, LossFunctions.hausdorff], [LossFunctions.accuracy, LossFunctions.f1]]

    if classification:
        net = encoder
        net.load_state_dict(torch.load(modelFile), strict=False)
    else:
        net = UNet(device, multiTask=False).to(device)
        net.load_state_dict(torch.load(modelFile), strict=False)

    #Evaluate Model
    print(f"Model: {modelName}")

    losses = TrainingEval.evaluate(net, testIter, lossFuncs, device=device, encoder=classification)
    logStr = ""
    for i, arr in enumerate(losses):
        for j, val in enumerate(arr):
            logStr += (lossFuncs[i][j].__name__ if str(type(lossFuncs[i][j])) == "<class 'function'>" else type(lossFuncs[i][j]).__name__) + ": " + str(val) + " "

    print(logStr)

In [ ]:
modelName = ""
modelFile = "UsedModels/ContrastiveTest1BestLoss"

net = ContrastiveEncoder().to(device)
net.load_state_dict(torch.load(modelFile), strict=False)

lossFunc = LossFunctions.ContrastiveLossCosine

loss = TrainingEval.contrastiveEval(net, contTestIter, lossFunc, device=device, isDist=False)
print(str(lossFunc) + " " + str(loss))

In [ ]:
#modelFile = "Run 2/Standard Pre-Training/PretrainedUNet7"
modelFile = "Run 2/Standard Pre-Training/PretrainedUNet6"
dataset = LITSBinaryDataset("Datasets/Scan1Dataset.hdf5")
iter = DataLoader(dataset, batch_size=batchSize)

net = UNet(0)
net.load_state_dict(torch.load(modelFile), strict=False)

net.to(device)

segmentationMask = TrainingEval.getMasks(net, iter, device=device)

masksFile = "PretrainMasksScan1"
wFile = h5py.File(masksFile, "w")

for i, slice in enumerate(segmentationMask):
    wFile.create_dataset("Slice" + str(i), data=slice)

wFile.close()

In [ ]:
#Close datasets
trainDataset.closeFile()
validationDataset.closeFile()
testDataset.closeFile()